In [1]:
from utils.imports import *

('/Volumes/solo/ali/Data/train_cls_20/train/false/', u'maked')
('/Volumes/solo/ali/Data/train_cls_20/train/true/', u'maked')


Using TensorFlow backend.


In [2]:
def create_cls_sample(df_anno,df_pred,img_file,data_path,output_true,output_false):
    mini_df_anno = df_anno[df_anno["file"]==img_file] #get all nodules associate with file
    if mini_df_anno.shape[0]>0: # some files may not have a nodule--skipping those 
        # load the data once
        patient_id = img_file.split('/')[-1][:-4]
        itk_img = SimpleITK.ReadImage(data_path + img_file) 
        img_array = SimpleITK.GetArrayFromImage(itk_img) # indexes are z,y,x (notice the ordering)
        num_z, height, width = img_array.shape        #heightXwidth constitute the transverse plane
        origin = np.array(itk_img.GetOrigin())      # x,y,z  Origin in world coordinates (mm)
        spacing = np.array(itk_img.GetSpacing())    # spacing of voxels in world coor. (mm)
        # go through all nodes (why just the biggest?)
        for node_idx, cur_row in mini_df_anno.iterrows():       
            node_x = cur_row["coordX"]
            node_y = cur_row["coordY"]
            node_z = cur_row["coordZ"]
            diam = cur_row["diameter_mm"]
            center = np.array([node_x, node_y, node_z])   # nodule center
            v_center = np.rint(np.absolute(center-origin)/spacing)  # nodule center in voxel space (still x,y,z ordering)
            img_array = normalize(img_array)
            
            new_x = int(v_center[0])
            new_y = int(v_center[1])
            new_z = int(v_center[2])
            trainX_1 =  img_array[new_x - 3: new_x + 3,
                                 new_y - 10 : new_y + 10,
                                 new_z - 10 : new_z + 10] 
            
            if trainX_1.shape == (6,20,20):
                np.save(output_true + str(patient_id)+'_'+str(node_idx)+'_'+str('_0_')+ '_3d_20_6_i.npy',trainX_1)            
               
        
                
    mini_df_pred = df_pred[df_pred["file"]==img_file] #get all nodules associate with file
    if mini_df_pred.shape[0]>0: # some files may not have a nodule--skipping those 
        # load the data once
        patient_id = img_file.split('/')[-1][:-4]
        itk_img = SimpleITK.ReadImage(data_path + img_file) 
        img_array = SimpleITK.GetArrayFromImage(itk_img) # indexes are z,y,x (notice the ordering)
        num_z, height, width = img_array.shape        #heightXwidth constitute the transverse plane
        origin = np.array(itk_img.GetOrigin())      # x,y,z  Origin in world coordinates (mm)
        spacing = np.array(itk_img.GetSpacing())    # spacing of voxels in world coor. (mm)
        # go through all nodes (why just the biggest?)
        for node_idx, cur_row in mini_df_pred.iterrows():       
            node_x = cur_row["coordX"]
            node_y = cur_row["coordY"]
            node_z = cur_row["coordZ"]
            diam = cur_row["diameter_mm"]
            center = np.array([node_x, node_y, node_z])   # nodule center
            v_center = np.rint(np.absolute(center-origin)/spacing)  # nodule center in voxel space (still x,y,z ordering)
            img_array = normalize(img_array)
            
            new_x = int(v_center[0])
            new_y = int(v_center[1])
            new_z = int(v_center[2])
            
            trainX =   img_array[new_x - 3: new_x + 3,
                                 new_y - 10: new_y + 10,
                                 new_z - 10: new_z + 10] 
    
            if trainX.shape == (6,20,20):
                np.save(output_false + str(patient_id)+'_'+str(node_idx)+'_'+str(new_y) + '_3d_20_6_i.npy',trainX)
    return

In [3]:
csv_path = PATH['annotations_train']
output_true = PATH['cls_train_20_true']
output_false = PATH['cls_train_20_false']
pred_csv_path = PATH['model_train_pred']
data_path = PATH['src_train']
anno_csv_new = pd.read_csv(csv_path + "annotations_all.csv")
pred_csv_new = pd.read_csv(pred_csv_path + "anno_false_final.csv")

In [4]:
pred_csv_new = pred_csv_new[pred_csv_new.index%12 == 0]

In [ ]:
patients=load_train(data_path)
anno_csv_new["file"] = anno_csv_new["seriesuid"].map(lambda file_name: get_filename(patients, file_name))
anno_csv_new = anno_csv_new.dropna()
pred_csv_new["file"] = pred_csv_new["seriesuid"].map(lambda file_name: get_filename(patients, file_name))
pred_csv_new = pred_csv_new.dropna()

In [ ]:
Parallel(n_jobs=-1)(delayed(create_cls_sample)(anno_csv_new,pred_csv_new,patient,data_path,output_true,output_false) for patient in tqdm(sorted(patients)))

 82%|████████▏ | 494/600 [24:52<04:04,  2.30s/it]  